# FiberSpace

An interactive plotting environment for visualizing structural parameters

In [1]:
import requests
import pandas as pd
import numpy as np
from skimage import io
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull
import bokeh
from bokeh.plotting import figure, show, output_notebook, output_file

from IPython.display import Image, HTML, Javascript
from coffeetools import coffee
output_notebook()

ImportError: No module named coffeetools

In [ ]:
# Must make all photos public on Flickr for this cell to work.

import flickrapi
api_key = u'e8366dedadcfc81dcd0c761492b91827'
api_secret = u'f8bc4e6c4e7bf4ff'

flickr = flickrapi.FlickrAPI(api_key, api_secret, format='json')
photos = flickr.photos.search(user_id='127699624@N02', per_page='100')
df_flickr = pd.read_json(photos)
df_flickr = pd.DataFrame(df_flickr['photos']['photo'])
df_flickr.to_csv('/Users/Imperssonator/CC/Zoom_Imgs/flickr.csv')
df_flickr.head()

In [3]:
df_flickr = pd.read_csv('/Users/Imperssonator/CC/Zoom_Imgs/flickr.csv')
df_flickr.head()

,Unnamed: 0,farm,id,isfamily,isfriend,ispublic,owner,secret,server,title
0,0,6,30642437092,0,0,1,127699624@N02,3d090af106,5738,MFU 0.25 spin coating 001_2um.002
1,1,6,30758994195,0,0,1,127699624@N02,821fd60309,5513,MFU 0.1 spin coating 001_2um.001
2,2,6,30642437132,0,0,1,127699624@N02,3c90407f91,5468,MFU 0.1 spin coating 002_2um.001
3,3,6,30758994255,0,0,1,127699624@N02,1e192c2be1,5465,MFU 0.6 spin coating 001_2um.001
4,4,6,30670369541,0,0,1,127699624@N02,37fed69b13,5482,SonAge Sol3 blade 2 V3.002 rot


In [4]:
# Flickr image url format:
# https://farm{farm-id}.staticflickr.com/{server-id}/{id}_{secret}.jpg

def make_url(img_path,df_flickr):
    row = df_flickr[df_flickr['title']==img_path].iloc[0]
    url = 'https://farm'+str(row['farm'])+'.staticflickr.com/'+ \
    str(row['server'])+'/'+str(row['id'])+ \
    '_'+str(row['secret'])+'.jpg'
    return url


In [5]:
import pandas as pd

the_csv_file = 'file:///Users/Imperssonator/Google Drive/My AFM/All Good Images/summary_stats_all.csv'
df = pd.read_csv(the_csv_file)
df.head()

,ImageName,Sfull,CorrLen,MeanLength,MeanWidth,LengthDensity,ImageSize,Process
0,MFU 0.1 spin coating 001_2um.001,0.873260,622.3635,361.6864,14.4767,0.015281,2,MFU
1,MFU 0.1 spin coating 002_2um.001,0.811830,402.2616,344.7610,15.1403,0.014652,2,MFU
2,MFU 0.25 spin coating 001_2um.002,0.337630,506.4571,164.7038,15.5168,0.011035,2,MFU
3,MFU 0.6 spin coating 001_2um.001,0.252630,495.0272,190.1355,14.5377,0.011836,2,MFU
4,10%_2min,0.087859,322.8267,148.7525,21.1618,0.009223,2,Son+2MP


In [6]:
df['file_url'] = df['ImageName'].apply(lambda x: make_url(x,df_flickr))
df['file_url'][10]

'https://farm6.staticflickr.com/5459/30721965636_f8d7e203bd.jpg'

In [7]:
colors = {}
class_to_color = 'Process'
for c in df[class_to_color].unique():
    colors[c] = np.random.randint( 0,256, (1,3))
    df.loc[df[class_to_color]==c,'color'] = '#' + ''.join([ '%02x'%s for s in colors[c][0]] )
    
df['color'].head()

0    #ba2475
1    #ba2475
2    #ba2475
3    #ba2475
4    #60ad78
Name: color, dtype: object

In [93]:
p2=figure(responsive=False, x_axis_label = 'S Full', y_axis_label = 'Average Fiber Length',width = 800, height=600)

# patches = []
# for c in df['class'].unique():
#     i = df.loc[df['class']==c].index.values
#     h = ConvexHull(df.iloc[i][['pca_x','pca_y']])
#     pp = p.patch(**{
#             'fill_color': df.iloc[i[0]].color,
#             'fill_alpha': .4,
#             'x': df.iloc[i].pca_x.values[h.vertices],
#             'y': df.iloc[i].pca_y.values[h.vertices],
#             'legend': c,
#         })
    
# gds=bokeh.models.ColumnDataSource(g)
# class_scale = p.oval( 
#     source=gds, 
#     x='pca_x.mean', 
#     width='pca_x.std',
#     y='pca_y.mean',
#     height='pca_y.std',
#     fill_color='color.min',
#     fill_alpha = .6,
# )

# p.add_tools(
#     bokeh.models.TapTool( 
#         renderers=[pp],
#         callback=bokeh.models.CustomJS(
#                 code=coffee.compile("""
#                 console.log 'click'
#                 """,bare=True)
#             )
#     )
# )

cds = bokeh.models.ColumnDataSource(
    df
)
# cds.data['volume_fraction'] = [ v*5 for v in cds.data['volume_fraction'] ]

circles = p2.circle(
    source=cds,
    x='Sfull',
    y='MeanLength',
    fill_color='color',
    size=20,
    fill_alpha=.6
)

p2.add_tools(
    bokeh.models.HoverTool(
        tooltips=[
            ('S full', '@Sfull'),
            ('Avg. Fiber Length (nm)','@MeanLength'),
            ('Image Size (um)','@ImageSize'),
            ('Process','@Process')
        ]
    )
)

images = p2.image_url(
    source=cds,
    x='Sfull',
    y='MeanLength',
    w=.01224,
    h=20,
    url='file_url',
    anchor='center',
    global_alpha=.001
)

# mu = [.5]
# std = np.sqrt(-1*np.diff(mu)**2/(2*np.log(.2)))

p2.x_range.callback = bokeh.models.CustomJS( args= {
        'x_axis': p2.x_range,
        'y_axis': p2.y_range,
        'images': images.glyph,
        'cds': cds,
    }, 
    code=coffee.compile("""
    dx = x_axis.get('end') - x_axis.get('start')
    
    ddata = Bokeh._.max(cds.get('data')['Sfull']) - Bokeh._.min(cds.get('data')['Sfull'])
    
    if dx > .0001 * ddata
        images.set 'global_alpha', .001 + (.8)*Math.exp( -1*Math.pow(dx-(.0001*ddata),2)/.014)
    """,bare=True)
)

show(p2)

In [62]:
p.axis[0].bounds

'auto'

In [83]:
output_file('FiberSpace.html')

In [8]:
show(p)

NameError: name 'p' is not defined